In [5]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, pairing
import galois
GF = galois.GF(curve_order)

In [39]:
import numpy as np
from functools import reduce
from py_ecc.bn128 import G1, G2, multiply, curve_order, neg, pairing, Z1, Z2
import py_ecc.bn128
import random


def decompose_polynomial(poly):
    # Replace minus signs with a '+' followed by the negative sign for splitting
    terms = poly.replace("-", "+-").split('+')

    # Normalize terms to ensure x^1 is written as x, x^0 as a constant
    terms = [t.strip().replace('x^1', 'x') for t in terms]

    equations = []
    counter = 1
    result = "result"
    result_is_zero = True

    def add_equation(lhs, rhs):
        nonlocal counter
        eq_name = f"v{counter}"
        equations.append(f"{eq_name} = {lhs} + {rhs}")
        counter += 1
        return eq_name

    def get_power_name():
        nonlocal counter
        name = f"v{counter}"
        counter += 1
        return name

    for term in terms:
        if 'x' in term:
            if '^' in term:
                coeff, power = term.split('x^')
                coeff = int(coeff) if coeff else 1
                power = int(power)
            else:
                coeff = term.split('x')[0]
                if coeff == "-":
                    coeff = -1
                else:
                    coeff = int(coeff) if coeff else 1
                power = 1

            current_x_power = "x"
            for i in range(2, power+1):
                new_power = get_power_name()
                equations.append(f"{new_power} = {current_x_power} * x")
                current_x_power = new_power

            if coeff != 1:
                coeff_power = get_power_name()
                equations.append(f"{coeff_power} = {coeff} * {current_x_power}")
                current_x_power = coeff_power

            if not result_is_zero:
                result = add_equation(result, current_x_power)
            else:
                result = current_x_power
                result_is_zero = False

        else:
            # For constant terms, simply add
            if not result_is_zero:
                result = add_equation(result, term.strip())
            else:
                result = term.strip()
                result_is_zero = False

    equations[-1] = f"out = {' '.join(equations[-1].split()[2:])}"

    words = []
    for equation in equations:
        tokens = equation.split()
        for token in reversed(tokens):
            if token.isnumeric() or (token.startswith("-") and token[1:].isnumeric()):
                words.append('1')
            elif token not in ["+", "*", "=", "-", "/"]:  # if the token is not a mathematical operator
                words.append(token)

    unique_words = []
    for elem in words:
        if elem not in unique_words:
            unique_words.append(elem)
    # Ensure '1' and 'out' are in the list and move them to indexes 0 and 1 respectively
    if '1' in unique_words:
        unique_words.remove('1')
        unique_words.insert(0, '1')
    if 'out' in unique_words:
        unique_words.remove('out')
        unique_words.insert(1, 'out')

    return (equations, list(unique_words))

def interpolate_lagrange(col, GF, num_eq):
    xs = GF(np.array([i for i in range(1, num_eq+1)]))
    poly = galois.lagrange_poly(xs, col)

    poly_return = np.pad(poly.coeffs, (max(num_eq - len(poly.coeffs), 0), 0), 'constant') #To have vectors of the same size
    return poly_return

def get_position_vector(unique_words, elements):
    # Ensure elements is a list
    if not isinstance(elements, list):
        elements = [elements]

    # Create a zero-filled list of length equal to unique_words
    vector = [0] * len(unique_words)

    for element in elements:
        # Attempt to convert the element to an integer, default to 1 if not possible
        try:
            value = int(element)
            element = '1'
        except ValueError:
            value = 1

        # Update the position in the vector by adding the deduced value if the element is in unique_words
        if element in unique_words:
            vector[unique_words.index(element)] += value

    return vector

def generate_r1cs(equations, ref_array):
    L = []
    R = []
    O = []
    for eq in equations:
        eq_split = eq.split()

        if eq_split[3] == '*':
            vecta = get_position_vector(ref_array, eq_split[2])
            L.append(vecta)
            vectb = get_position_vector(ref_array, eq_split[4])
            R.append(vectb)
            vectc = get_position_vector(ref_array, eq_split[0])
            O.append(vectc)

        if eq_split[3] == '+':
            if "out" in eq:
                list_var = eq_split[2:]
            else:
                list_var = [eq_split[2], eq_split[4]]
            vecta = get_position_vector(ref_array, list_var)
            L.append(vecta)
            vectb = get_position_vector(ref_array, "1")
            R.append(vectb)
            vectc = get_position_vector(ref_array, eq_split[0])
            O.append(vectc)

    return (L, R, O)

def evaluate_expression(expr, values_dict):
    terms = expr.split()
    if '+' in terms:
        idx = terms.index('+')
        left_expr = " ".join(terms[:idx])
        right_expr = " ".join(terms[idx+1:])
        left_val = evaluate_expression(left_expr, values_dict)
        right_val = evaluate_expression(right_expr, values_dict)
        if isinstance(left_val, str):
            return left_val + " + " + str(right_val)
        elif isinstance(right_val, str):
            return str(left_val) + " + " + right_val
        else:
            return left_val + right_val
    elif '*' in terms:
        idx = terms.index('*')
        left_expr = " ".join(terms[:idx])
        right_expr = " ".join(terms[idx+1:])
        left_val = evaluate_expression(left_expr, values_dict)
        right_val = evaluate_expression(right_expr, values_dict)
        if isinstance(left_val, str):
            return left_val + " * " + str(right_val)
        elif isinstance(right_val, str):
            return str(left_val) + " * " + right_val
        else:
            return left_val * right_val
    else:
        if terms[0] in values_dict:
            return values_dict[terms[0]]
        elif terms[0].isnumeric():
            return int(terms[0])  # Convert string number to integer
        else:
            return expr



def compute_solution_vector(unique_words, equations, values):
    values_dict = {'1': 1}
    values_dict.update(values)

    unresolved_equations = equations.copy()

    while unresolved_equations:
        newly_resolved = []

        for equation in unresolved_equations:
            lhs, rhs = equation.split("=")
            lhs = lhs.strip()
            rhs_value = evaluate_expression(rhs.strip(), values_dict)

            if not isinstance(rhs_value, str):  # if the evaluation is successful
                values_dict[lhs] = rhs_value
                newly_resolved.append(equation)
            else:
                print(f"{lhs} remains unresolved.")

        if not newly_resolved:  # if no new equations were resolved in this pass
            print("Cannot resolve:", unresolved_equations)
            break

        for equation in newly_resolved:
            unresolved_equations.remove(equation)

    return [values_dict.get(word, 0) for word in unique_words]


def evaluate_polynomial_galois(coefs, x, GF):
    result = GF(0)
    for coef in coefs:
        result = result * x + GF(coef)
    return int(result)

def inner_product_polynomials_with_witness(polys, witness):
        mul_ = lambda x, y: x * y
        sum_ = lambda x, y: x + y
        return reduce(sum_, map(mul_, polys, witness))


In [40]:
def trusted_setup(U,V,W,l):
 
    m=len(U)-1
    n=len(U[0])

    #========================== Pour trouver t(x)  ==========================
    values = [i for i in range(1, len(U[0])+1)]
    poly_t = galois.Poly([1], field=GF)
    for val in values:
        poly_t *= galois.Poly([1, -val], field=GF)
    t=[]
    for elem in poly_t.coeffs:
        t.append(int(elem))
    #=========================================================================
    
    x=GF(random.randint(1,curve_order-1))
    print("x = ",int(x))

    t_x=evaluate_polynomial_galois(t, x, GF)

    
    # gamma =GF(random.randint(1,curve_order-1))
    # delta =GF(random.randint(1,curve_order-1))

    # print("alpha = ",int(alpha))
    # print("beta = ",int(beta))
    # print("gamma = ",int(gamma))
    # print("delta = ",int(delta))

    x_power_i=[]
    for i in range(0, n):
        x_power_i.append(x**i)

    x_power_i_t_x=[]
    for i in range(0,n-2+1):#[0,n-1]
        x_power_i_t_x.append((x**i)*t_x)


    G1_x_power_i = [multiply(G1,int(elem)) for elem in x_power_i]
    G1_x_power_i_t_x = [multiply(G1,int(elem)) for elem in x_power_i_t_x]
    G2_x_power_i = [multiply(G2,int(elem)) for elem in x_power_i]

    return (G1_x_power_i, G1_x_power_i_t_x, G2_x_power_i)

def prover(U,V,W,l,param,a):
    print("\n============= Prover =============")

    r=GF(random.randint(1,curve_order-1))
    s=GF(random.randint(1,curve_order-1))

    m=len(U)-1
    n=len(U[0]) 

    G1_x_power_i, G1_x_power_i_t_x, G2_x_power_i = param

    #=================================Pour trouver t(x)  ==========================
    values = [i for i in range(1, len(U[0])+1)] # Car on a 6 colonnes
    poly_t = galois.Poly([1], field=GF)  # Start with the polynomial '1' in the given field
    for val in values:
        poly_t *= galois.Poly([1, -val], field=GF)
    t=[]
    for elem in poly_t.coeffs:
        t.append(int(elem))
    #=================================Pour trouver h(x)  ==========================
    Uw = galois.Poly(np.matmul(np.transpose(U), a))
    Vw = galois.Poly(np.matmul(np.transpose(V), a))
    Ww = galois.Poly(np.matmul(np.transpose(W), a))
    t_poly=galois.Poly(t,field=GF)

    h = (Uw * Vw - Ww) // t_poly
    h_rem = (Uw * Vw - Ww) % t_poly

    assert h_rem == 0, "h(x) is not a multiple of t(x)"

    h_t_x=None
    for i in range(0, len(h.coeffs)):
        h_t_x=py_ecc.bn128.add(h_t_x,multiply(G1_x_power_i_t_x[i],int(h.coeffs[::-1][i])))

    #==============================================================================

    Ui_x = []

    for i in range(0, m+1):
        temp=None
        for j in range(0, len(U[i])):
            temp2=multiply(G1_x_power_i[j],int(U[i][::-1][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        Ui_x.append(temp)
        
    A_1=None
    for i in range(0, m+1):
        temp=multiply(Ui_x[i],int(a[i]))
        A_1=py_ecc.bn128.add(A_1,temp)
    print("A_1=",A_1)

    V_i_x_2 = []
    for i in range(0, m+1):
        temp=None
        for j in range(0, len(V[i])):
            temp2=multiply(G2_x_power_i[j],int(V[i][::-1][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        V_i_x_2.append(temp)

    B_2=None
    for i in range(0, m+1):
        temp=multiply(V_i_x_2[i],int(a[i]))
        B_2=py_ecc.bn128.add(B_2,temp)
    print("B_2=",B_2)

    W_i_x_1 = []
    for i in range(0, m+1):
        temp=None
        for j in range(0, len(W[i])):
            temp2=multiply(G1_x_power_i[j],int(W[i][::-1][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        W_i_x_1.append(temp)

    C_1=None
    for i in range(0, m+1):
        temp=multiply(W_i_x_1[i],int(a[i]))
        C_1=py_ecc.bn128.add(C_1,temp)
    C_1 = py_ecc.bn128.add(C_1, h_t_x)

    print("C_1=",C_1)
    return A_1, B_2, C_1

def verifier(pi,a,param,l):
    print("\n============= Verifier =============")

    A_1, B_2, C_1 = pi

    G1_x_power_i, G1_x_power_i_t_x, G2_x_power_i = param

    pairing1 = pairing(B_2, A_1)
    pairing2 = pairing(G2, C_1)

    # print("pairing1 = ",pairing1)
    # print("pairing2 = ",pairing2)

    return pairing1==pairing2



In [41]:
import numpy as np
import random

O = np.array([
[0,0,0,1,0,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1],
[0,1,0,0,0,0,0,0]
])
L = np.array([
[ 0,0,1,0,0,0,0,0],
[ 0,0,1,0,0,0,0,0],
[ 3,0,0,0,0,0,0,0],
[ 5,0,0,0,0,0,0,0],
[10,0,0,0,0,0,0,0],
[ 3,0,0,0,0,1,1,1]
])
R = np.array([
[0,0,1,0,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,1,0,0,0,0,0],
[1,0,0,0,0,0,0,0]
])

# # Circuit initial
# v1 = x * x
# v2 = x * v1
# v3 = 3 * v2
# v4 = 5 * v1
# v5 = 10 * x
# out = v3 + v4 + v5 + 3

w=[1, 553, 5, 25, 125, 375, 125, 50] # pour etre comme dans le papier
result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))

assert result.all(), "Le produit ne fonctionne pas"
print(" → Vecteur choisi", w)

 → Vecteur choisi [1, 553, 5, 25, 125, 375, 125, 50]


In [42]:
def interpolate_lagrange(col, GF, num_eq):
    xs = GF(np.array([i for i in range(1, num_eq+1)]))
    poly = galois.lagrange_poly(xs, col)
    poly_return = np.pad(poly.coeffs, (max(num_eq - len(poly.coeffs), 0), 0), 'constant') #To have vectors of the same size
    return poly_return

num_eq = len(L)

p=curve_order

L_galois = GF(np.array(L) % p)
R_galois = GF(np.array(R) % p)
O_galois = GF(np.array(O) % p)

U = []
V = []
W = []

for i in range(len(w)):
    U.append(interpolate_lagrange(L_galois[:, i], GF, num_eq))
    V.append(interpolate_lagrange(R_galois[:, i], GF, num_eq))
    W.append(interpolate_lagrange(O_galois[:, i], GF, num_eq))

print("Matrix U:")
for line in U:
    for elem in line:
        print(elem, end=" ")
    print()

print("\nMatrix V:")
for line in V:
    for elem in line:
        print(elem, end=" ")
    print()

print("\nMatrix W:")
for line in W:
    for elem in line:
        print(elem, end=" ")
    print()
    
w=GF(w)



Matrix U:
9302503220531691969454722441734341912633054870176814596071736779294718610637 6384070837619788606488535009033371900826606283454676683578642887751277477892 2736030358979909402780800718157159386068545550052004292962275523321976061929 4560050598299849004634667863595265643447575916753340488270459205536626769988 20793830728247311461134085457994411334120946180395232626513293977247018070746 42 
0 0 0 0 0 0 
12403337627375589292606296588979122550177406493569086128095649039059624814183 13680151794899547013904003590785796930342727750260021464811377616609880309760 12768141675239577212977070018066743801653212566909353367157285775502554955781 8208091076939728208342402154471478158205636650156012878886826569965928185842 18605006441063383938909444883468683825266109740353629192143473558589437221295 21888242871839275222246405745257275088548364400416034343698204186575808495608 
0 0 0 0 0 0 
0 0 0 0 0 0 
8572895124803716128713175583559099409681442723496280117948463306408858327450 2736030358979909

In [43]:
U=GF(U)
V=GF(V)
W=GF(W)

l = 1 # Publicly provide the first 2 elements of a, the rest is private

param= trusted_setup(U, V, W, l)
pi = prover(U, V, W, l, param, w)
isTrue = verifier(pi, w, param, l)
print(isTrue)

x =  11634552059110308648693737517861129381811915041561174938422691485176812077964



============= Prover =============
A_1= (21284014151541815793131666911200284445789017827329536373249190673160361877533, 9118083106311707895246603038264754131473673181782714910440862561370692487586)
B_2= ((16540226607715467805257043643118700100081194517302847467967689386926611564876, 11675214832496828287399210101728337608185254599407534390632121832852204891754), (9350403625643744358332080188988674492139989324238008534250351454484795777960, 14580541693372992223087907566948990087962372129274098145712136820811068089191))
C_1= (16860414719142082783217215882716000820350547301455516970658273206900057729017, 8548780026967937875070227138419006567257507642329747693026736364770432964080)

============= Verifier =============
True
